In [ ]:
#Simple Neural network with one hidden layer to train to predict Cat.
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [ ]:
def initialization_deep(layer_dims):
    parameters={}
    np.random.seed(3)
    for l in range(1,len(layer_dims)):
        parameters['W'+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])*np.sqrt(2/layer_dims[l-1])
        parameters['b'+str(l)]=np.zeros((layer_dims[l],1))
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))    
    return parameters    

In [ ]:
def relu(Z):
    A=np.maximum(0,Z)
    cache=Z
    return A,cache
def sigmoid(Z):
    A=1/(1+np.exp(-Z))
    cache=Z
    return A,cache

In [ ]:
def relu_backward(dA,cache):
    Z=cache
    dZ=np.array(dA,copy=True)
    dZ[Z<=0]=0
    return dZ
def sigmoid_backward(dA,cache):
    Z=cache
    s=1/(1+np.exp(-Z))
    dZ=dA*s*(1-s)
    return dZ

In [ ]:
def linear_forward(A,W,b):
    Z=W@A+b
    cache=(A,W,b)
    return Z,cache

In [ ]:
def linear_activation_forward(A_prev,W,b,activation):
    Z,linear_cache=linear_forward(A_prev,W,b)
    if activation=="relu":
        A,activation_cache=relu(Z)
    elif activation=="sigmoid":
        A,activation_cache=sigmoid(Z)
    cache=(linear_cache,activation_cache)
    return A,cache

In [ ]:
def model_forward(X,parameters):
    caches=[]
    A=X
    L=len(parameters)//2
    for l in range(1,L):
        A_prev=A
        A,cache=linear_activation_forward(A_prev,parameters['W'+str(l)],parameters['b'+str(l)],activation="relu")
        caches.append(cache)
    AL,cache=linear_activation_forward(A,parameters['W'+str(L)],parameters['b'+str(L)],activation="sigmoid")
    caches.append(cache)
    return AL,caches

In [ ]:
def compute_cost(AL,Y):
    m=Y.shape[1]
    cost=-1/m*(Y@np.log(AL).T+((1-Y)@np.log(1-AL).T))
    return cost

In [ ]:
def compute_cost_with_regularization(A3,Y,parameters,lambd):
    m=Y.shape[1]
    W1=parameters["W1"]
    W2=parameters["W2"]
    W3=parameters["W3"]
    cross_entropy_cost=compute_cost(A3,Y)
    L2_regularization_cost = (1/m)*((lambd/2)*(np.sum(np.square(W1))+np.sum(np.square(W2))+np.sum(np.square(W3))))
    cost = cross_entropy_cost + L2_regularization_cost
    return cost

In [ ]:
def linear_backward(dZ,cache):
    A_prev,W,b=cache
    m=A_prev.shape[1]
    dW=1/m*dZ@A_prev.T
    db=1/m*np.sum(dZ,axis=1,keepdims=True)
    dA_prev=W.T@dZ
    return dA_prev,dW,db

In [ ]:
def linear_activation_backward(dA,cache,activation):
    linear_cache,activation_cache=cache
    if activation=="relu":
        dZ=relu_backward(dA,activation_cache)
        dA_prev,dW,db=linear_backward(dZ,linear_cache)
    else:# activation=="sigmoid":
        dZ=sigmoid_backward(dA,activation_cache)
        dA_prev,dW,db=linear_backward(dZ,linear_cache)
    return dA_prev,dW,db    

In [ ]:
def L_model_backward(AL,Y,caches):
    grads={}
    L=len(caches)
    m=AL.shape[1]
    Y=Y.reshape(AL.shape)
    dAL=-(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    current_cache=caches[L-1]
    grads["dA"+str(L-1)],grads["dW"+str(L)],grads["db"+str(L)]=linear_activation_backward(dAL,current_cache,activation="sigmoid")
    for l in reversed(range(L-1)):
        current_cache=caches[l]
        dA_prev_temp,dW_temp,db_temp=linear_activation_backward(grads["dA"+str(l+1)],caches[l],activation="relu")
        grads["dA" +str(l)]=dA_prev_temp
        grads["dW" +str(l+1)]=dW_temp
        grads["db"+str(l+1)]=db_temp
    return grads    

In [ ]:
def update(parameters,grads,learning_rate):
    L=len(parameters)//2
    for l in range(L):
        parameters["W"+str(l+1)]-=learning_rate*grads["dW"+str(l+1)]
        parameters["b"+str(l+1)]-=learning_rate*grads["db"+str(l+1)]
    return parameters    

In [ ]:
def load_data():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
train_x_orig, train_y, test_x_orig, test_y, classes=load_data()

In [ ]:
plt.imshow(train_x_orig[13])


In [ ]:
m_train=train_x_orig.shape[0]
num_px=train_x_orig.shape[1]
m_test=test_x_orig.shape[0]
print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_x_orig shape: " + str(train_x_orig.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x_orig shape: " + str(test_x_orig.shape))
print ("test_y shape: " + str(test_y.shape))

In [ ]:
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

In [ ]:
layers_dims=[12288,20,7,5,1]

In [ ]:
def L_layer_model(X,Y,layers_dims,learning_rate=0.0075,num_iterations=3000,print_cost=False):
    np.random.seed(1)
    costs=[]
    parameters=initialization_deep(layers_dims)
    for i in range(0,num_iterations):
        AL,caches=model_forward(X,parameters)
        cost=compute_cost(AL,Y)
        grads=L_model_backward(AL,Y,caches)
        parameters=update(parameters,grads,learning_rate)
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations(per hundred)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    return parameters
        

In [ ]:
def predict(X,y,parameters):
    m=X.shape[1]
    n=len(parameters)
    p=np.zeros((1,m))
    probas,caches=model_forward(X,parameters)
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    print("Accuracy: "  + str(np.sum((p == y)/m)))
    return p
    

In [ ]:
parameteres=L_layer_model(train_x,train_y,layers_dims,learning_rate=0.0075,num_iterations=2500,print_cost=True)

In [ ]:
pred_train=predict(train_x,train_y,parameteres)

In [ ]:
pred_test=predict(test_x,test_y,parameteres)